In [3]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import Normalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Flatten, LSTM
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Lambda

# Data & Targets

In [4]:
df = pd.read_csv('/Users/nhuphuong/code/sweetpatata/bike-or-nei/raw_data/data_all_cleaned.csv')
df = df.drop(columns=['Trips_in','Trips_out', 'Unnamed: 0'])

In [3]:
print(df.shape)
df.columns

(293762, 27)


Index(['Station_Id', 'Date', 'In_Out', 'temp_min', 'temp_max',
       'wind_speed_avg', 'rainfall_total', 'snow_total', 'day_of_week_0',
       'day_of_week_1', 'day_of_week_2', 'day_of_week_3', 'day_of_week_4',
       'day_of_week_5', 'day_of_week_6', 'Month_1', 'Month_2', 'Month_3',
       'Month_4', 'Month_5', 'Month_6', 'Month_7', 'Month_8', 'Month_9',
       'Month_10', 'Month_11', 'Month_12'],
      dtype='object')

In [4]:
TARGET = 'In_Out'
N_TARGETS = 1
N_FEATURES = 24

# Extracting Data for one station

In [5]:
df.head()

,Unnamed: 0,Station_Id,Date,Trips_in,Trips_out,In_Out,temp_min,temp_max,wind_speed_avg,rainfall_total,...,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12
0,183879,556,2019-04-02,1,1,0,-2.04,11.21,4.633333,0.00,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,377,2019-04-03,12,15,-3,-2.79,6.61,4.620833,1.71,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,177199,550,2019-04-03,6,11,-5,-2.79,6.61,4.620833,1.71,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,178251,551,2019-04-03,68,40,28,-2.79,6.61,4.620833,1.71,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,179453,552,2019-04-03,22,17,5,-2.79,6.61,4.620833,1.71,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
len(df.groupby('Date'))

1269

In [7]:
df_station_377 = df[df['Station_Id'] == 377]

In [8]:
df_station_377.shape

(1203, 30)

In [9]:
df_station_377 = df_station_377.drop(columns=['Trips_in','Trips_out', 'Unnamed: 0', 'Station_Id'])

In [10]:
df_station_377.set_index('Date', inplace=True)

In [49]:
df_station_377.dtypes

In_Out              int64
temp_min          float64
temp_max          float64
wind_speed_avg    float64
rainfall_total    float64
snow_total        float64
day_of_week_0     float64
day_of_week_1     float64
day_of_week_2     float64
day_of_week_3     float64
day_of_week_4     float64
day_of_week_5     float64
day_of_week_6     float64
Month_1           float64
Month_2           float64
Month_3           float64
Month_4           float64
Month_5           float64
Month_6           float64
Month_7           float64
Month_8           float64
Month_9           float64
Month_10          float64
Month_11          float64
Month_12          float64
dtype: object

# Folds for cross-validation

In [12]:
FOLD_LENGTH = 30 # total days 1269 / total months 43
FOLD_STRIDE = 14
TRAIN_TEST_RATIO = 0.66

In [5]:
def get_folds(
    df,
    fold_length,
    fold_stride):
    '''
    This function slides through the Time Series dataframe of shape (n_timesteps, n_features) to create folds
    - of equal `fold_length`
    - using `fold_stride` between each fold
    
    Returns a list of folds, each as a DataFrame
    '''
    
    folds = []
    for idx in range(0, len(df), fold_stride):
        # Exits the loop as soon as the last fold index would exceed the last index
        if (idx + fold_length) > len(df):
            break
        fold = df.iloc[idx:idx + fold_length, :]
        folds.append(fold)#(np.asarray(fold).astype('float32'))
    return folds

In [53]:
folds = get_folds(df_station_377, FOLD_LENGTH, FOLD_STRIDE)

print(f'The function generated {len(folds)} folds.')
print(f'Each fold has a shape equal to {folds[0].shape}.')

The function generated 84 folds.
Each fold has a shape equal to (30, 25).


In [15]:
input_length = 5
output_length = 1

# Test-train split (one fold)

In [6]:
def train_test_split(fold,
                     train_test_ratio,
                     input_length):
    '''
    Returns a train dataframe and a test dataframe (fold_train, fold_test)
    from which one can sample (X,y) sequences.
    df_train should contain all the timesteps until round(train_test_ratio * len(fold))   
    '''
    # TRAIN SET
    # ======================
    last_train_idx = round(train_test_ratio * len(fold))
    fold_train = fold.iloc[0:last_train_idx, :]

    # TEST SET
    # ======================    
    first_test_idx = last_train_idx - input_length
    fold_test = fold.iloc[first_test_idx:, :]

    return (fold_train, fold_test)

In [17]:
(fold_train, fold_test) = train_test_split(folds[0], TRAIN_TEST_RATIO, input_length)

In [7]:
def get_Xi_yi(
    fold, 
    input_length, 
    output_length):
    '''
    - given a fold, it returns one sequence (X_i, y_i)
    - with the starting point of the sequence being chosen at random
    '''
    first_possible_start = 0
    last_possible_start = len(fold) - (input_length + output_length) + 1
    random_start = np.random.randint(first_possible_start, last_possible_start)
    X_i = fold.iloc[random_start:random_start+input_length]    
    y_i = fold.iloc[random_start+input_length:
                  random_start+input_length+output_length][[TARGET]]
    
    return (X_i, y_i)

In [59]:
X_train_i, y_train_i = get_Xi_yi(fold_train, input_length, output_length)
X_test_i, y_test_i = get_Xi_yi(fold_test, input_length, output_length)

In [8]:
def get_X_y(
    fold,
    number_of_sequences,
    input_length,
    output_length
    ):
    
    X, y = [], []

    for i in range(number_of_sequences):
        (Xi, yi) = get_Xi_yi(fold, input_length, output_length)
        X.append(Xi)
        y.append(yi)
        
    return np.array(X), np.array(y)

In [21]:
N_TRAIN = 60 # number_of_sequences_train
N_TEST =  40 # number_of_sequences_test

X_train, y_train = get_X_y(fold_train, N_TRAIN, input_length, output_length)
X_test, y_test = get_X_y(fold_test, N_TEST, input_length, output_length)

In [36]:
X_test.shape

(40, 5, 25)

In [22]:
#for fold in folds:
#    fold_train, fold_test = train_test_split(fold, TRAIN_TEST_RATIO, input_length)
#    return fold 

# RNN Model (one fold)

In [9]:
def init_model(X_train, y_train):
    
    # $CHALLENGIFY_BEGIN    
    
    # 0 - Normalization
    # ======================    
    normalizer = Normalization()
    normalizer.adapt(X_train)
    
    # 1 - RNN architecture
    # ======================    
    model = Sequential()
    ## 1.0 - All the rows will be standardized through the already adapted normalization layer
    model.add(normalizer)
    ## 1.1 - Recurrent Layer
    model.add(LSTM(64, 
                          activation='tanh', 
                          return_sequences = False,
                          recurrent_dropout = 0.3))
    ## 1.2 - Predictive Dense Layers
    output_length = y_train.shape[1]
    model.add(Dense(output_length, activation='linear'))

    # 2 - Compiler
    # ======================    
    adam = optimizers.Adam(learning_rate=0.02)    
    model.compile(loss='mse', optimizer=adam, metrics=["mae"])
    
    return model

In [10]:
def fit_model(model, verbose=1):

    # $CHALLENGIFY_BEGIN
    es = EarlyStopping(monitor = "val_loss",
                      patience = 3,
                      mode = "min",
                      restore_best_weights = True)


    history = model.fit(X_train, y_train,
                        validation_split = 0.3,
                        shuffle = False,
                        batch_size = 32,
                        epochs = 50,
                        callbacks = [es],
                        verbose = verbose)

    return model, history

In [26]:
# 1 - Initialising the RNN model
# ====================================

model = init_model(X_train, y_train)
model.summary()

# 2 - Training
# ====================================
model, history = fit_model(model)

2022-12-01 16:21:03.572387: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, None, 25)         51        
 n)                                                              
                                                                 
 lstm (LSTM)                 (None, 64)                23040     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 23,156
Trainable params: 23,105
Non-trainable params: 51
_________________________________________________________________
Epoch 1/50
2/2 [==============================] - 2s 266ms/step - loss: 18.7985 - mae: 3.2741 - val_loss: 8.0692 - val_mae: 2.2104
Epoch 2/50
2/2 [==============================] - 0s 18ms/step - loss: 13.2107 - mae: 2.7495 - val_loss: 7.6796 - val_

In [27]:
res = model.evaluate(X_test, y_test)

2/2 [==============================] - 0s 2ms/step - loss: 64.4005 - mae: 7.2036


# Baseline Model (for one fold)

In [28]:
def init_baseline():

    model = Sequential()
    model.add(Lambda(lambda x: x[:,-1,1,None]))

    adam = optimizers.Adam(learning_rate=0.02)
    model.compile(loss='mse', optimizer=adam, metrics=["mae"])

    return model

In [29]:
baseline_model = init_baseline()
baseline_score = baseline_model.evaluate(X_test, y_test)

2/2 [==============================] - 0s 1ms/step - loss: 130.8991 - mae: 8.3885


# Cross-validation (all folds)

In [54]:
def cross_validate_baseline_and_lstm():
    '''
    This function cross-validates 
    - the "last seen value" baseline model
    - the RNN model
    '''
    
    list_of_mae_baseline_model = []
    list_of_mae_recurrent_model = []
    
    # 0 - Creating folds
    # =========================================    
    folds = get_folds(df_station_377, FOLD_LENGTH, FOLD_STRIDE)
    
    for fold_id, fold in enumerate(folds):
        
        # 1 - Train/Test split the current fold
        # =========================================
        (fold_train, fold_test) = train_test_split(fold, TRAIN_TEST_RATIO, input_length)                   

        X_train, y_train = get_X_y(fold_train, N_TRAIN, input_length, output_length)
        X_test, y_test = get_X_y(fold_test, N_TEST, input_length, output_length)
        
        # 2 - Modelling
        # =========================================
        
        ##### Baseline Model
        baseline_model = init_baseline()
        
        print(type(X_test[0,0,0]))
        
        mae_baseline = baseline_model.evaluate(X_test, y_test, verbose=0)[1]
        list_of_mae_baseline_model.append(mae_baseline)
        print("-"*50)
        print(f"MAE baseline fold n°{fold_id} = {round(mae_baseline, 2)}")

        ##### LSTM Model
        model = init_model(X_train, y_train)
        es = EarlyStopping(monitor = "val_mae",
                           mode = "min",
                           patience = 2, 
                           restore_best_weights = True)
        history = model.fit(X_train, y_train,
                            validation_split = 0.3,
                            shuffle = False,
                            batch_size = 32,
                            epochs = 50,
                            callbacks = [es],
                            verbose = 0)
        res = model.evaluate(X_test, y_test, verbose=0)
        mae_lstm = res[1]
        list_of_mae_recurrent_model.append(mae_lstm)
        print(f"MAE LSTM fold n°{fold_id} = {round(mae_lstm, 2)}")
        
        ##### Comparison LSTM vs Baseline for the current fold
        print(f"🏋🏽‍♂️ improvement over baseline: {round((1 - (mae_lstm/mae_baseline))*100,2)} % \n")

    return list_of_mae_baseline_model, list_of_mae_recurrent_model

In [87]:
mae_list

[2.079939365386963,
 2.7373292446136475,
 2.183718204498291,
 13.436391830444336,
 4.472980976104736,
 7.5922532081604,
 6.412288665771484,
 2.9414238929748535,
 13.4185209274292,
 4.35196590423584,
 3.3075268268585205,
 7.084357261657715,
 4.481198787689209,
 3.043718099594116,
 2.8073012828826904,
 4.4678802490234375,
 3.1924681663513184,
 3.1045684814453125,
 7.727598667144775,
 4.149720668792725,
 3.5571722984313965,
 2.620628833770752,
 3.82080340385437,
 6.596154689788818,
 3.2003378868103027,
 4.790660381317139,
 9.98023509979248,
 4.216279029846191,
 6.139786243438721,
 7.0306549072265625,
 8.084578514099121,
 2.001192569732666,
 1.7682855129241943,
 12.33507251739502,
 2.7202868461608887,
 3.6841368675231934,
 2.7075271606445312,
 3.4725775718688965,
 8.158315658569336,
 2.88555908203125,
 2.2327446937561035,
 2.9615254402160645,
 2.284156322479248,
 3.839937686920166,
 13.681694030761719,
 4.059967994689941,
 2.563697338104248,
 3.753520965576172,
 8.57215690612793,
 7.209758

In [92]:
pred_list = []
for i in df_list:
                      

    X_train, y_train = get_X_y(i, N_TRAIN, input_length, output_length)
    X_test, y_test = get_X_y(i, N_TEST, input_length, output_length)

    prediction_test = model.predict(X_test[10])
    
    pred_list.append(int(prediction_test))

1/1 [==============================] - 0s 7ms/step


In [93]:
pred_list

[-1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,


In [55]:
mae_baselines, mae_lstms = cross_validate_baseline_and_lstm()

<class 'numpy.float64'>
--------------------------------------------------
MAE baseline fold n°0 = 8.86
MAE LSTM fold n°0 = 7.29
🏋🏽‍♂️ improvement over baseline: 17.69 % 

<class 'numpy.float64'>
--------------------------------------------------
MAE baseline fold n°1 = 7.95
MAE LSTM fold n°1 = 6.1
🏋🏽‍♂️ improvement over baseline: 23.3 % 

<class 'numpy.float64'>
--------------------------------------------------
MAE baseline fold n°2 = 14.84
MAE LSTM fold n°2 = 9.35
🏋🏽‍♂️ improvement over baseline: 37.02 % 

<class 'numpy.float64'>
--------------------------------------------------
MAE baseline fold n°3 = 14.97
MAE LSTM fold n°3 = 17.76
🏋🏽‍♂️ improvement over baseline: -18.68 % 

<class 'numpy.float64'>
--------------------------------------------------
MAE baseline fold n°4 = 19.9
MAE LSTM fold n°4 = 12.59
🏋🏽‍♂️ improvement over baseline: 36.74 % 

<class 'numpy.float64'>
--------------------------------------------------
MAE baseline fold n°5 = 12.79
MAE LSTM fold n°5 = 5.35
🏋🏽‍♂️ i

MAE LSTM fold n°39 = 3.09
🏋🏽‍♂️ improvement over baseline: -24.2 % 

<class 'numpy.float64'>
--------------------------------------------------
MAE baseline fold n°40 = 1.84
MAE LSTM fold n°40 = 2.4
🏋🏽‍♂️ improvement over baseline: -30.26 % 

<class 'numpy.float64'>
--------------------------------------------------
MAE baseline fold n°41 = 11.02
MAE LSTM fold n°41 = 1.67
🏋🏽‍♂️ improvement over baseline: 84.83 % 

<class 'numpy.float64'>
--------------------------------------------------
MAE baseline fold n°42 = 10.12
MAE LSTM fold n°42 = 1.45
🏋🏽‍♂️ improvement over baseline: 85.68 % 

<class 'numpy.float64'>
--------------------------------------------------
MAE baseline fold n°43 = 17.25
MAE LSTM fold n°43 = 4.62
🏋🏽‍♂️ improvement over baseline: 73.22 % 

<class 'numpy.float64'>
--------------------------------------------------
MAE baseline fold n°44 = 1.97
MAE LSTM fold n°44 = 2.48
🏋🏽‍♂️ improvement over baseline: -25.96 % 

<class 'numpy.float64'>
---------------------------------

MAE LSTM fold n°78 = 1.53
🏋🏽‍♂️ improvement over baseline: 85.55 % 

<class 'numpy.float64'>
--------------------------------------------------
MAE baseline fold n°79 = 11.36
MAE LSTM fold n°79 = 1.93
🏋🏽‍♂️ improvement over baseline: 83.05 % 

<class 'numpy.float64'>
--------------------------------------------------
MAE baseline fold n°80 = 7.37
MAE LSTM fold n°80 = 2.24
🏋🏽‍♂️ improvement over baseline: 69.63 % 

<class 'numpy.float64'>
--------------------------------------------------
MAE baseline fold n°81 = 6.98
MAE LSTM fold n°81 = 3.89
🏋🏽‍♂️ improvement over baseline: 44.35 % 

<class 'numpy.float64'>
--------------------------------------------------
MAE baseline fold n°82 = 6.18
MAE LSTM fold n°82 = 2.77
🏋🏽‍♂️ improvement over baseline: 55.17 % 

<class 'numpy.float64'>
--------------------------------------------------
MAE baseline fold n°83 = 3.55
MAE LSTM fold n°83 = 3.39
🏋🏽‍♂️ improvement over baseline: 4.57 % 



In [56]:
print(f"average percentage improvement over baseline = {round(np.mean(1 - (np.array(mae_lstms)/np.array(mae_baselines))),2)*100}%")

average percentage improvement over baseline = 31.0%


# Prediction for one station on date x

In [62]:
X_test.shape

(40, 5, 25)

In [63]:
prediction_test = model.predict(X_test[0])

1/1 [==============================] - 0s 162ms/step


In [68]:
int(prediction_test)

-4

In [66]:
y_test[0]

array([[-15]])

# 9 Prediction for all stations

In [11]:
df_dict = {}
for station in df['Station_Id'].unique():
    df_station = df[df['Station_Id'] == station]
    df_station.set_index('Date', inplace=True)
    df_station = df_station.drop(columns='Station_Id')
    df_dict["{}".format(station)] = df_station

### Spilt each station_df into train and test data

In [12]:
N_TRAIN = 60 # number_of_sequences_train
N_TEST =  40 # number_of_sequences_test
input_length = 5
output_length = 1

In [15]:
TARGET = 'In_Out'
N_TARGETS = 1
N_FEATURES = 24
LOCAL_REGISTRY_PATH = '/Users/nhuphuong/code/sweetpatata/bike-or-nei/training_output'

In [16]:
X_train_dict = {}
y_train_dict = {}
X_test_dict = {}
y_test_dict = {}
for station, df_station in df_dict.items():
    X_train, y_train = get_X_y(df_station, N_TRAIN, input_length, output_length)
    X_test, y_test = get_X_y(df_station, N_TEST, input_length, output_length)
    X_train_dict["X_train_{}".format(station)] = X_train
    y_train_dict["y_train_{}".format(station)] = y_train
    X_test_dict["X_test_{}".format(station)] = X_test
    y_test_dict["y_test_{}".format(station)] = y_test

In [ ]:
for (X_train_station, df_X_train), (y_train_station, df_y_train) in zip(X_train_dict.items(), y_train_dict.items()):
    X_train = df_X_train
    y_train = df_y_train
    model = init_model(X_train, y_train)
    model, history = fit_model(model)
    save_model(model)

Epoch 1/50


2022-12-02 10:49:16.575196: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-02 10:49:16.591240: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-02 10:49:17.403741: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 2s 532ms/step - loss: 6.9063 - mae: 1.8054 - val_loss: 7.5257 - val_mae: 1.9841
Epoch 2/50


2022-12-02 10:49:18.301372: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 0s 135ms/step - loss: 3.6885 - mae: 1.2782 - val_loss: 7.5427 - val_mae: 2.1451
Epoch 3/50
2/2 [==============================] - 0s 109ms/step - loss: 2.6805 - mae: 1.1290 - val_loss: 8.9580 - val_mae: 2.3758
Epoch 4/50
2/2 [==============================] - 0s 135ms/step - loss: 1.9659 - mae: 0.9513 - val_loss: 11.9924 - val_mae: 2.7236
- model path: /Users/nhuphuong/code/sweetpatata/bike-or-nei/training_output/models/X_train_556
INFO:tensorflow:Assets written to: /Users/nhuphuong/code/sweetpatata/bike-or-nei/training_output/models/X_train_556/assets

✅ data saved locally
Epoch 1/50


...

2/2 [==============================] - 5s 2s/step - loss: 212.6187 - mae: 10.4474 - val_loss: 373.8763 - val_mae: 15.2125
Epoch 2/50
2/2 [==============================] - 1s 636ms/step - loss: 171.3752 - mae: 9.2399 - val_loss: 294.1931 - val_mae: 13.3220
Epoch 3/50
2/2 [==============================] - 1s 476ms/step - loss: 123.6929 - mae: 7.1350 - val_loss: 195.0531 - val_mae: 10.0438
Epoch 4/50
2/2 [==============================] - 1s 378ms/step - loss: 84.5527 - mae: 5.6141 - val_loss: 141.3949 - val_mae: 8.9115
Epoch 5/50
2/2 [==============================] - 0s 228ms/step - loss: 70.3933 - mae: 5.6654 - val_loss: 123.3809 - val_mae: 8.8356
Epoch 6/50
2/2 [==============================] - 1s 397ms/step - loss: 67.1533 - mae: 5.6567 - val_loss: 114.4743 - val_mae: 8.5535
Epoch 7/50
2/2 [==============================] - 1s 315ms/step - loss: 64.8446 - mae: 5.5475 - val_loss: 114.4149 - val_mae: 8.6458
Epoch 8/50
2/2 [==============================] - 0s 157ms/step - loss: 59.6

2022-12-02 11:26:07.647200: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-02 11:26:07.663853: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Epoch 1/50


2022-12-02 11:26:08.774646: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.



1/2 [==============>...............] - ETA: 2s - loss: 10.0334 - mae: 2.0037

In [21]:
import glob
import os
import time
import pickle

from colorama import Fore, Style

from tensorflow.keras import Model, models

In [75]:
def save_model(model: Model = None,
               params: dict = None,
               metrics: dict = None) -> None:
    """
    persist trained model, params and metrics
    """

    #timestamp = time.strftime("%Y%m%d-%H%M%S")

    #print(Fore.BLUE + "\nSave model to local disk..." + Style.RESET_ALL)

    # save params
    if params is not None:
        params_path = os.path.join(LOCAL_REGISTRY_PATH, "params", X_train_station + ".pickle")
        print(f"- params path: {params_path}")
        with open(params_path, "wb") as file:
            pickle.dump(params, file)

    # save metrics
    if metrics is not None:
        metrics_path = os.path.join(LOCAL_REGISTRY_PATH, "metrics", X_train_station + ".pickle")
        print(f"- metrics path: {metrics_path}")
        with open(metrics_path, "wb") as file:
            pickle.dump(metrics, file)

    # save model
    if model is not None:
        model_path = os.path.join(LOCAL_REGISTRY_PATH, "models", X_train_station)
        print(f"- model path: {model_path}")
        model.save(model_path)

    print("\n✅ data saved locally")

    return None


In [53]:
def load_model(X_new) -> Model:
    """
    load the latest saved model, return None if no model found
    """
    
    station_list = []
    pred_list = []
    # get latest model version
    model_directory = os.path.join(LOCAL_REGISTRY_PATH, "models")

    results = glob.glob(f"{model_directory}/*")
    if not results:
        return None
    
    for model_path in results:
        print(f"- path: {model_path}")
        station_name = model_path[-3:]
        station_list.append(station_name)
        model = models.load_model(model_path)
        pred_list.append(round(model.predict(X_new)[0][0],0))
        
    pred_df = pd.DataFrame(zip(station_list, pred_list))

    return pred_df


In [54]:
%time
pred_df = load_model(X_test_dict['X_test_556'][0])
pred_df

CPU times: user 4 µs, sys: 2 µs, total: 6 µs
Wall time: 34.1 µs
- path: /Users/nhuphuong/code/sweetpatata/bike-or-nei/training_output/models/X_train_460
1/1 [==============================] - 0s 265ms/step


2022-12-02 14:37:40.815887: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


- path: /Users/nhuphuong/code/sweetpatata/bike-or-nei/training_output/models/X_train_494
1/1 [==============================] - 0s 206ms/step
- path: /Users/nhuphuong/code/sweetpatata/bike-or-nei/training_output/models/X_train_458


2022-12-02 14:37:43.194383: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 240ms/step


...

- path: /Users/nhuphuong/code/sweetpatata/bike-or-nei/training_output/models/X_train_424
1/1 [==============================] - 0s 206ms/step
- path: /Users/nhuphuong/code/sweetpatata/bike-or-nei/training_output/models/X_train_616


2022-12-02 14:45:42.918991: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 236ms/step


2022-12-02 14:45:44.849158: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


,0,1
0,460,1.0
1,494,47.0
2,458,1.0
3,493,0.0
4,451,1.0
...,...,...
230,415,-3.0
231,611,28.0
232,423,2.0
233,424,3.0


In [55]:
pred_df

,0,1
0,460,1.0
1,494,47.0
2,458,1.0
3,493,0.0
4,451,1.0
...,...,...
230,415,-3.0
231,611,28.0
232,423,2.0
233,424,3.0


# Predict some days and save local

In [61]:
X_new = pd.read_csv('/Users/nhuphuong/code/sweetpatata/bike-or-nei/raw_data/test_pred.csv')
X_new = X_new.drop(columns = ['Unnamed: 0'])

In [65]:
X_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   temp_min        6 non-null      float64
 1   temp_max        6 non-null      float64
 2   wind_speed_avg  6 non-null      float64
 3   snow_total      6 non-null      int64  
 4   rainfall_total  6 non-null      int64  
 5   day_of_week_0   6 non-null      float64
 6   day_of_week_1   6 non-null      float64
 7   day_of_week_2   6 non-null      float64
 8   day_of_week_3   6 non-null      float64
 9   day_of_week_4   6 non-null      float64
 10  day_of_week_5   6 non-null      float64
 11  day_of_week_6   6 non-null      float64
 12  Month_1         6 non-null      float64
 13  Month_2         6 non-null      float64
 14  Month_3         6 non-null      float64
 15  Month_4         6 non-null      float64
 16  Month_5         6 non-null      float64
 17  Month_6         6 non-null      float64

In [66]:
pred_df_0212 = load_model(X_new.iloc[0])
pred_df_0212

- path: /Users/nhuphuong/code/sweetpatata/bike-or-nei/training_output/models/X_train_460


2022-12-02 15:07:28.161136: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 542ms/step
- path: /Users/nhuphuong/code/sweetpatata/bike-or-nei/training_output/models/X_train_494


2022-12-02 15:07:30.698723: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 300ms/step
- path: /Users/nhuphuong/code/sweetpatata/bike-or-nei/training_output/models/X_train_458


...

2022-12-02 15:15:54.379560: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 308ms/step
- path: /Users/nhuphuong/code/sweetpatata/bike-or-nei/training_output/models/X_train_616


2022-12-02 15:15:56.558446: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 311ms/step


,0,1
0,460,0.0
1,494,46.0
2,458,-2.0
3,493,-1.0
4,451,-5.0
...,...,...
230,415,0.0
231,611,37.0
232,423,-2.0
233,424,0.0


In [ ]:
pred_df_0312 = load_model(X_new.iloc[1])
pred_df_0312

- path: /Users/nhuphuong/code/sweetpatata/bike-or-nei/training_output/models/X_train_460


2022-12-02 15:15:58.568573: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 351ms/step
- path: /Users/nhuphuong/code/sweetpatata/bike-or-nei/training_output/models/X_train_494


2022-12-02 15:16:00.765697: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 317ms/step
- path: /Users/nhuphuong/code/sweetpatata/bike-or-nei/training_output/models/X_train_458


...

1/1 [==============================] - 0s 343ms/step
- path: /Users/nhuphuong/code/sweetpatata/bike-or-nei/training_output/models/X_train_496


2022-12-02 15:21:15.653735: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 363ms/step
- path: /Users/nhuphuong/code/sweetpatata/bike-or-nei/training_output/models/X_train_462


2022-12-02 15:21:17.920821: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 358ms/step
- path: /Users/nhuphuong/code/sweetpatata/bike-or-nei/training_output/models/X_train_465


In [ ]:
pred_df_0412 = load_model(X_new.iloc[2])
pred_df_0412

In [ ]:
pred_df_0512 = load_model(X_new.iloc[3])
pred_df_0512

In [ ]:
pred_df_0612 = load_model(X_new.iloc[4])
pred_df_0612

In [28]:
model_path = results[0]

In [39]:
%time
pred_dict = {}
station_name = model_path[-3:]
model = models.load_model(model_path)
pred_dict["{}".format(station_name)] = round(model.predict(X_test_dict['X_test_556'][0])[0][0],0)

CPU times: user 5 µs, sys: 11 µs, total: 16 µs
Wall time: 31 µs
1/1 [==============================] - 0s 161ms/step


2022-12-02 12:48:49.400194: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [42]:
pred_dict

{'460': 1.0}

In [56]:
X_test_dict['X_test_556'][0].shape

(5, 25)

In [57]:
X_test_dict['X_test_556'][0]

array([[ 1.        ,  1.58      ,  6.74      ,  9.1275    ,  8.82      ,
         1.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        , -1.01      , 11.5       ,  4.82833333,  0.2       ,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 1.        , -2.51      , 13.8       ,  3.75      ,  0.4       ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  1.        ,  0